<a href="https://colab.research.google.com/github/upasana-crypto/EEG/blob/main/chb08_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# New Section

In [2]:
!pip install pyedflib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.4 MB 33.7 MB/s 


In [3]:
import pyedflib
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import os
import warnings 
import random
import collections
from PIL import Image
from numpy.random import seed
import datetime
from time import mktime
from time import gmtime, strptime


In [4]:
import warnings
warnings.filterwarnings("ignore")
pathDataSet = '/content/drive/MyDrive/upsi-seizure/data/chb08/'
patients = ["08"]

In [5]:
def loadSummaryPatient(index):
  f = open(pathDataSet+'/chb'+patients[index]+'-summary (1).txt', 'r') #change to summary.txt
  parent = '/content/drive/MyDrive/upsi-seizure/data/chb'+patients[index]+'/'
  return f, parent

In [7]:
def createDataset():

    print("START \n")
    for indexPatient in range(0, len(patients)):
        # fileList = []
    
        f, parent = loadSummaryPatient(indexPatient)
        line=f.readline()
        while (line):
            data=line.split(':')
            if (data[0]=="File Name"):
                name_F=data[1].strip()
                print(name_F)
                fileSt=f.readline().split(': ')[1]
                Sthr=fileSt.split(':')[0]
                print(Sthr)
                Stmin=fileSt.split(':')[1]
                print(Stmin)
                Stsec=fileSt.split(':')[2]
                print(Stsec)
                fileEn=f.readline().split(': ')[1]
                Enhr=fileEn.split(':')[0]
                print(Enhr)
                Enmin=fileSt.split(':')[1]
                print(Enmin)
                Ensec=fileSt.split(':')[2]
                print(Ensec)
                trial_duration = ((int(Enhr) - int(Sthr))*3600 + (int(Enmin) - int(Stmin))*60 + (int(Ensec) - int(Stsec)))
                for i in range(1):
                    line=f.readline()
                for j in range(0, int(line.split(': ')[1])):
                    secSt=int(f.readline().split(': ')[1].split(' ')[0])
                    secEn=int(f.readline().split(': ')[1].split(' ')[0])
                    seizureImageGenerate(secSt, secEn, name_F, parent)
                break   # remove this
            line=f.readline()
        f.close()

    print("END \n")

In [8]:
def seizureImageGenerate(secSt, secEn, name_F, parent):
  file1 = pyedflib.EdfReader(pathDataSet+name_F)
  n = file1.signals_in_file
  # print(n)
  signal_labels = file1.getSignalLabels()
  signal_headers = file1.getSignalHeaders()
  rate = signal_headers[0]['sample_rate']
  dur = file1.getFileDuration()
  x = np.zeros((n, file1.getNSamples()[0]))
  for i in range(n):
    x[i,:] = file1.readSignal(i)
    # print(x)
    label = file1.getLabel(i)
  file1.close()
  #x_filter = butter_bandpass_filter(x ,lowcut , highcut , fs , order = 5)
  #a = os.getcwd()
  path = '/content/drive/MyDrive/upsi-seizure/test/'
  if os.path.isdir(path) is not True:
    os.makedirs(path)
  picnum = int(dur*rate/256)
  c1 =0
  c2 = 0
  for i in range(picnum):
    img = x[:,i*256:(i+1)*256]
    img = (img - np.min(img))/(np.max(img) - np.min(img))
    #Img = Image.fromarray(np.uint8(img))
    if secSt <= i+1 <= secEn: #window size is 1sec
      filename = '_time_seizure_'+ str(i+1)
      np.save(path + name_F.split('.')[0] + filename+'.npy', img)
      #print('hi')
      c1+=1
    else:
      filename = '_time_nonseizure_'+ str(i+1)
      np.save(path + name_F.split('.')[0] + filename+'.npy', img)
      #print('no=hi')
      c2+=1

  print(c1+c2)

In [9]:
createDataset()

START 

chb08_02.edf
12
28
57

13
28
57

3600
END 



In [6]:
from tensorflow.keras.utils import Sequence
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, roc_curve, auc, fbeta_score

In [18]:
def generatePathList(patients, val_size, test_size):
    parent_path = '/content/drive/MyDrive/upsi-seizure/test/'
    pathList = []
    for indexPatient in range(0, len(patients)):
        sub_path = 'chb'+patients[indexPatient]+'/'
        directory_name = parent_path
        for filename in os.listdir(directory_name):
            pathList.append(directory_name+filename)
    L = len(pathList)
    train_index_size = int(L*(1-val_size-test_size))
    val_index_size = int(L*val_size)
    test_index_size = int(L*test_size)
    train_index = random.sample(pathList, train_index_size)
    val_index = random.sample(pathList, val_index_size)
    test_index = random.sample(pathList, test_index_size)
    print(test_index)
    return pathList.index(train_index), pathList(val_index), pathList.index(test_index), pathList

class DataGenerator(Sequence):

    def __init__(self, index, pathList, parent_path = '/content/drive/MyDrive/upsi-seizure/test/', batch_size=32):
        self.batch_size = batch_size
        self.parent_path = parent_path
        self.pathList = pathList
        self.index = index
        self.L = len(self.index)


    def __len__(self):
        return self.L - self.batch_size

    def __getitem__(self, idx):
        batch_indexs = self.index[idx:(idx+self.batch_size)]
        image_path = [self.pathList[k] for k in batch_indexs]
        #print(batch_indexs)
        #print(image_path)
        return self._load_image(image_path)

  
    def _load_image(self, image_path):
        features = np.zeros(((len(image_path)),23,256))
        labels = np.zeros((len(image_path)),dtype=int)
        i = 0 #the feature index
        for name in image_path:
            #print(name)
            if '_seizure_' in name:
                features[i] = np.array(np.load(name))[0:23,:]
                labels[i] = 1
                #print('hi')
            elif '_nonseizure_' in name:
                features[i] = np.array(np.load(name))[0:23,:]
                labels[i] = 0
            i = i+1
        #print(features)
        #print(labels)
        #print(np.expand_dims(np.array(features), axis=3).shape)
        #print(labels.shape)
        #print('features', features)
        print('labels', labels)
        #print(features.shape)
        s = np.expand_dims(np.array(features), axis=3)
        #print(s.shape)
        return np.expand_dims(np.array(features), axis=3),labels

In [9]:
import os

print(len(os.listdir(r"/content/drive/MyDrive/upsi-seizure/test")))

3600


In [10]:
import tensorflow
from tensorflow import keras 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Dense, Activation, Flatten, LSTM, Bidirectional, ConvLSTM2D
from tensorflow.keras.layers import Conv2D, MaxPooling2D,GlobalAveragePooling2D, BatchNormalization, TimeDistributed, SimpleRNN, GRU
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.backend import clear_session

In [11]:
model = Sequential()

model.add(Conv2D(4, (1, 8), strides = (1,1), padding = 'same', input_shape=((23, 256, 1))))
#model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(1, 8), strides=(1,8)))
#model.add(Dropout(0.15))

model.add(Conv2D(16, (1, 16),strides = (1,1), padding = 'same'))
#model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(1, 4), strides=(1,4)))
#model.add(Dropout(0.15))

model.add(Conv2D(16, (1, 8),strides = (1,1), padding = 'same'))
#model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(1, 4), strides=(1,4)))
#model.add(Dropout(0.15))

model.add(Conv2D(16, (16, 1),strides = (1,1), padding = 'same'))
#model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(4, 1), strides=(4,1)))
#model.add(Dropout(0.15))

model.add(Conv2D(16, (16, 1),strides = (1,1), padding = 'same'))
#model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(4, 1), strides=(4,1)))
#model.add(Dropout(0.15))

model.add(GlobalAveragePooling2D())

model.add(Dense(1, activation='sigmoid'))

sgd = SGD(learning_rate=0.0001, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(metrics=['accuracy',keras.metrics.Recall(name='sen'), keras.metrics.SpecificityAtSensitivity(0.98)], 
              loss='binary_crossentropy', optimizer= 'sgd')

In [19]:
import os
import random
import collections
train_id, val_id, test_id, path_list = generatePathList(patients,val_size= 0.3, test_size=0.1) # 30% test_size 
#print(test_id)
#print(train_id)
train_data = DataGenerator(train_id, path_list)
test_data  =  DataGenerator(test_id, path_list)

['/content/drive/MyDrive/upsi-seizure/test/chb08_02_time_nonseizure_531.npy', '/content/drive/MyDrive/upsi-seizure/test/chb08_02_time_nonseizure_2102.npy', '/content/drive/MyDrive/upsi-seizure/test/chb08_02_time_nonseizure_426.npy', '/content/drive/MyDrive/upsi-seizure/test/chb08_02_time_nonseizure_909.npy', '/content/drive/MyDrive/upsi-seizure/test/chb08_02_time_nonseizure_2012.npy', '/content/drive/MyDrive/upsi-seizure/test/chb08_02_time_seizure_2722.npy', '/content/drive/MyDrive/upsi-seizure/test/chb08_02_time_nonseizure_2568.npy', '/content/drive/MyDrive/upsi-seizure/test/chb08_02_time_nonseizure_3427.npy', '/content/drive/MyDrive/upsi-seizure/test/chb08_02_time_nonseizure_1645.npy', '/content/drive/MyDrive/upsi-seizure/test/chb08_02_time_nonseizure_1773.npy', '/content/drive/MyDrive/upsi-seizure/test/chb08_02_time_nonseizure_2457.npy', '/content/drive/MyDrive/upsi-seizure/test/chb08_02_time_nonseizure_3292.npy', '/content/drive/MyDrive/upsi-seizure/test/chb08_02_time_nonseizure_46

ValueError: ignored

In [78]:
#test_data.__getitem__(100)

In [13]:
print(len(train_data))
print(len(test_data))

1048
2128


In [32]:
import numpy as np

In [16]:
history = model.fit_generator(generator=train_data, epochs= 2, verbose=1, steps_per_epoch=None, 
                    validation_data=test_data, class_weight={0:1, 1:20}, workers=4, use_multiprocessing=True,
                    shuffle=False, initial_epoch=0)

TypeError: ignored

In [63]:
y_true= [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
print(y_true)
print(y_pred)

[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0.17241904139518738, 0.9545454382896423, 0.6899086833000183]


In [62]:
confusion_matrix(y_true, y_pred)

ValueError: ignored

In [36]:
print(history)

In [66]:
y_pred = model.evaluate(test_data, batch_size = 32)

[1111, 3155, 1138, 16148, 5963, 11390, 925, 8960, 15675, 1890, 4611, 16218, 12759, 2456, 9913, 14993, 3668, 17427, 10137, 665, 2628, 3781, 12491, 5656, 9605, 8445, 10252, 2246, 2192, 6386, 17392, 10180]
['/content/drive/MyDrive/upsi-seizure/normalised/chb08_02_time_nonseizure_1112.npy', '/content/drive/MyDrive/upsi-seizure/normalised/chb08_02_time_nonseizure_3156.npy', '/content/drive/MyDrive/upsi-seizure/normalised/chb08_02_time_nonseizure_1139.npy', '/content/drive/MyDrive/upsi-seizure/normalised/chb08_21_time_nonseizure_1749.npy', '/content/drive/MyDrive/upsi-seizure/normalised/chb08_05_time_nonseizure_2364.npy', '/content/drive/MyDrive/upsi-seizure/normalised/chb08_13_time_nonseizure_591.npy', '/content/drive/MyDrive/upsi-seizure/normalised/chb08_02_time_nonseizure_926.npy', '/content/drive/MyDrive/upsi-seizure/normalised/chb08_11_time_nonseizure_1761.npy', '/content/drive/MyDrive/upsi-seizure/normalised/chb08_21_time_nonseizure_1276.npy', '/content/drive/MyDrive/upsi-seizure/norma

ValueError: ignored